In [1]:
import os

import gdi_loader as gdi
import bert_swiss_lm as bsl
import time

In [2]:
# Setup loader
loader = gdi.GdiLoader(set_path="/Users/raphael/Downloads/gdi-vardial-2019/")
# Set up the two BERT embedders
bert_swiss_lm_embedder = bsl.BertSwissLm("/Users/raphael/Downloads/bert-swiss-lm")
swiss_bert_embedder = bsl.BertSwissLm("ZurichNLP/swissbert", set_default_language_to_de_CH=True)
# Load GDI data
train_data = loader.create_dataframe(which_type="train")
dev_data = loader.create_dataframe(which_type="dev")
test_data = loader.create_dataframe(which_type="gold")

Some weights of the model checkpoint at /Users/raphael/Downloads/bert-swiss-lm were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ZurichNLP/swissbert were not used when initializing XmodModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.wei

In [3]:
# Embed data twice, once with the BERT model from Michael Jungo and once with SwissBERT
# This takes approximately 1 hour
start = time.time()
train_embedding_df = bert_swiss_lm_embedder.add_sentence_embedding_to_df(train_data)
dev_embedding_df = bert_swiss_lm_embedder.add_sentence_embedding_to_df(dev_data)
test_embedding_df = bert_swiss_lm_embedder.add_sentence_embedding_to_df(test_data)

train_swissbert_embedding_df = swiss_bert_embedder.add_sentence_embedding_to_df(train_data)
dev_swissbert_embedding_df = swiss_bert_embedder.add_sentence_embedding_to_df(dev_data)
test_swissbert_embedding_df = swiss_bert_embedder.add_sentence_embedding_to_df(test_data)
print(time.time() - start)

12.329530000686646


In [4]:
# Save the embeddings to disk for later use
if not os.path.exists("data"):
    os.mkdir("data")
train_embedding_df.to_feather("data/train_embedding_bert_swiss_lm.feather")
dev_embedding_df.to_feather("data/dev_embedding_bert_swiss_lm.feather")
test_embedding_df.to_feather("data/test_embedding_bert_swiss_lm.feather")

train_swissbert_embedding_df.to_feather("data/train_embedding_swissbert.feather")
test_swissbert_embedding_df.to_feather("data/test_embedding_swissbert.feather")
dev_swissbert_embedding_df.to_feather("data/dev_embedding_swissbert.feather")
